Pseudo arc-length continuation in the CR3BP
=====================================

In this example we will refine the basic continuation scheme used in [Continuation of Periodic Orbits in the CR3BP](<./Periodic orbits in the CR3BP.ipynb>) and use instead, as continuation parameter, the pseudo arc-length.

The resulting numerical scheme allows to follow through folds of the curve implicitly defined by the periodicity condition and the Poincare' phasing condition.

## Preamble
As usual, we make some standard imports:

In [1]:
import heyoka as hy
import numpy as np
import time 

from scipy.optimize import least_squares
from copy import deepcopy

from matplotlib.pylab import plt

... and define some functions that will help later on to visualize our trajectories and make nice plots. (ignore them and come back to this later in case you are curious)

In [2]:
def compute_L_points(mu, f):
    """Computes The exact position of the Lagrangian points. To do so it finds the zeros of the
    the dynamics equation for px.
    
        Args:
            mu (float): The value of the mu parameter.
            f (heyoka expression): The px dynamics equation.

        Returns:
            xL1, xL2, xL3, xL45, yL45: The coordinates of the various Lagrangian Points
    """
    # Position of the lagrangian points approximated
    xL1 = (mu-1) + (mu/3/(1-mu))**(1/3)
    xL2 = (mu-1) - (mu/3/(1-mu))**(1/3)
    xL3 = -(mu-1) - 7/12 * mu / (1-mu)
    yL45 = np.sin(60/180*np.pi)
    xL45 = -0.5 + mu

    # Solve for the static equilibrium from the approximated solution
    def equilibrium(expr, x,y):
        retval = hy.eval(expr, {"x":x, "y":y, "z":0, "px":-y, "py":x, "pz":0}, [mu])
        return retval
    xL1 = root_scalar(lambda x: equilibrium(f, x,0.), x0=xL1,x1=xL1-1e-2).root
    xL2 = root_scalar(lambda x: equilibrium(f, x,0.), x0=xL2,x1=xL2-1e-2).root
    xL3 = root_scalar(lambda x: equilibrium(f, x,0.), x0=xL3,x1=xL3-1e-2).root;
    return xL1, xL2, xL3, xL45, yL45

def potential_function(position,mu):
    """Computes the system potential
        Args:
            position (array-like): The position in Cartesian coordinates
            mu (float): The value of the mu parameter.

        Returns:
            The potential
    """
    x,y,z=position
    r_1=np.sqrt((x-mu)**2+y**2+z**2)
    r_2=np.sqrt((x-mu+1)**2+y**2+z**2)
    Omega=1./2.*(x**2+y**2)+(1-mu)/r_1+mu/r_2
    return Omega

def jacobi_constant(state,mu):
    """Computes the system Jacobi constant
        Args:
            state (array-like): The system state (x,y,z,px,py,pz)
            mu (float): The value of the mu parameter.

        Returns:
            The Jacobi constant for the state
    """
    x,y,z,px,py,pz=state
    vx = px + y
    vy = py - x
    vz = pz
    r_1=np.sqrt((x-mu)**2+y**2+z**2)
    r_2=np.sqrt((x-mu+1)**2+y**2+z**2)
    Omega=1/2*(x**2+y**2)+(1-mu)/r_1+mu/r_2
    T=1/2*(vx**2+vy**2+vz**2)
    C=Omega-T
    return C

We also define the CR3BP equations ... (see [circular restricted three-body problem](<./The restricted three-body problem.ipynb>)) 

In [3]:
# Create the symbolic variables.
symbols_state = ["x", "y", "z", "px", "py", "pz"]
x = np.array(hy.make_vars(*symbols_state))
# This will contain the r.h.s. of the equations
f = []

rps_32 = ((x[0] - hy.par[0])**2 + x[1]**2 + x[2]**2)**(-3/2.)
rpj_32 = ((x[0] - hy.par[0]  + 1.)**2 + x[1]**2 + x[2]**2)**(-3/2.)

# The equations of motion.
f.append(x[3] + x[1])
f.append(x[4] - x[0])
f.append(x[5])
f.append(x[4] - (1. - hy.par[0]) * rps_32 * (x[0] - hy.par[0]) - hy.par[0] * rpj_32 * (x[0] - hy.par[0] + 1.))
f.append(-x[3] -((1. - hy.par[0]) * rps_32 + hy.par[0] * rpj_32) * x[1])
f.append(-((1. - hy.par[0]) * rps_32 + hy.par[0] * rpj_32) * x[2])
f = np.array(f)

and the corresponding variational equations (see and [Continuation of Periodic Orbits in the CR3BP](<./Periodic orbits in the CR3BP.ipynb>))), essentially computing the state transition matrix $\mathbf \Phi$.

In [4]:
symbols_phi = []
for i in range(6):
    for j in range(6):
        # Here we define the symbol for the variations
        symbols_phi.append("phi_"+str(i)+str(j))  
phi = np.array(hy.make_vars(*symbols_phi)).reshape((6,6))

In [5]:
dfdx = []
for i in range(6):
    for j in range(6):
        dfdx.append(hy.diff(f[i],x[j]))
dfdx = np.array(dfdx).reshape((6,6))

In [6]:
# The (variational) equations of motion
dphidt = dfdx@phi

Finally, we create the dynamics including all 6 + 6x6 = 42 equations:

In [7]:
dyn = []
for state, rhs in zip(x,f):
    dyn.append((state, rhs))
for state, rhs in zip(phi.reshape((36,)),dphidt.reshape((36,))):
    dyn.append((state, rhs))
# These are the initial conditions on the variational equations (the identity matrix)
ic_var = np.eye(6).reshape((36,)).tolist()

and instantiate the Taylor integrator (high accuracy and no compact mode)

In [8]:
start_time = time.time()
ta = hy.taylor_adaptive(
    # The ODEs.
    dyn,
    # The initial conditions.
    [-0.45, 0.80, 0.00, -0.80, -0.45, 0.58] + ic_var,
    # Operate below machine precision
    # and in high-accuracy mode.
    tol = 1e-18, high_accuracy = True
)
print("--- %s seconds --- to build the Taylor integrator" % (time.time() - start_time))

--- 11.609195470809937 seconds --- to build the Taylor integrator


## The Pseudo arc-length continuation method
In the most general form, and in a nutshell, numerical continuation considers a starting equation in the form:
$$
\underline{{\mathbf G}}\left(\underline{ \mathbf x}, \lambda\right) = \underline{\mathbf 0}
$$
assumes a solution $\underline{ \mathbf x}_0, \lambda_0$ is available and seeks a new solution $\underline{ \mathbf x}_0+\delta\underline{\mathbf{x}}$ corrisponding to $\lambda_0+\delta \lambda$. In cases where the solution curve implicitly defined by $\underline{{\mathbf G}}$ folds with respect to the continuation parameter $\lambda$, this becomes clearly problematic. A common solution is to consider a new continuation parameter intrinsically linked to the solution curve geometry: the curve arc-length $s$. Assuming a reference $s=s_0$ (can be zero) in $\underline{ \mathbf x}_0, \lambda_0$, the arc-length becomes a function of $\underline{\mathbf{x}}$ and $\lambda$. We can then write $s = \tilde s\left(\underline{ \mathbf x}, \lambda\right)$.

Under this idea one can formally write:
$$
\left\{
\begin{array}{l}
\underline{{\mathbf G}}\left(\underline{ \mathbf x}, \lambda\right) = \underline{\mathbf 0} \\
\tilde s\left(\underline{ \mathbf x}, \lambda\right) - s = 0
\end{array}
\right.
$$
or, equivalently:
$$
\underline{{\mathbf G}}^*\left(\underline{ \mathbf y}, s\right) = \underline{\mathbf 0}
$$
where $\underline{ \mathbf y} = [\underline{ \mathbf y}, s ]$ and $\underline{\mathbf G}^*=[\underline{\mathbf G}, \tilde s - s]$. Starting from the solution $\underline{ \mathbf x}_0, \lambda_0, s_0$ we have reformulated the problem and may now seek a new solution $[\underline{ \mathbf x}_0+\delta\underline{\mathbf{x}}, \lambda_0+\delta \lambda]$ corresponding to $s_0+\delta s$. We obtain (at least formally) a continuation scheme able to follow easily through folds of the $\underline{ \mathbf x}, \lambda$ curve, since the $\underline{ \mathbf y}, s$ curve has none! (the arc-length is always uniformly increasing along the solution curve).

The problem now is that we cannot really compute $\tilde s$ easily! But we can approximate $\delta s$ as the projection of $[\delta \underline{ \mathbf x}, \delta \lambda]$ onto the tangent vector $\mathbf \tau = \left[\frac{d\underline{ \mathbf x}}{ds}, \frac{d\lambda}{ds}\right]$ (hence the name **pseudo** arc-length is used). Eventually, the system:
$$
\left\{
\begin{array}{l}
\underline{{\mathbf G}}\left(\underline{ \mathbf x}, \lambda\right) = \underline{\mathbf 0} \\
\mathbf \tau \cdot [\delta \underline{ \mathbf x}, \delta \lambda] +  s_0 - s = 0
\end{array}
\right.
$$
is considered and its solutions continued, now using $s$ as continuation parameter.

### The case of continuing periodic orbits.
In the case of a search for periodic solutions to the system $\dot{\underline{\mathbf x}} =\underline{ \mathbf f}(\underline{\mathbf x})$ we indicate the generic solution as $\underline{ \mathbf x}\left(t; \underline{ \mathbf x}_0\right)$ and may thus write the periodicity condition as:
$$
(1) \qquad \underline{ \mathbf x}\left(T; \underline{ \mathbf x}_0\right) - \underline{ \mathbf x}_0 = \mathbf 0
$$
We thus search for $\underline{ \mathbf x}_0, T$ roots of the equation above. Assuming we have one such solution, we will slightly abuse the notation and omit a further $0$ as a subscript in $\underline{\mathbf x}_0$ indicating such an initial solution simply as as $\underline{\mathbf x}_0, T_0$. To continue this initial solution we seek $\delta\underline{\mathbf x}$ and $\delta T$ so that:
$$
\underline{ \mathbf x}\left(T_0 + \delta T; \underline{ \mathbf x}_0 + \delta\underline{\mathbf x}\right) - \underline{ \mathbf x}_0 - \delta\underline{\mathbf x}= \mathbf 0
$$
In other words we are perturbing our initial condition and period and demand to end up in a new periodic orbit. It is straight forward to see that for any fixed $\delta T$ there is more than one $\underline{ \mathbf x}_0$ that would do the trick. Infact any point on the whole new periodic orbit would work! Thus, as we saw also in [Continuation of Periodic Orbits in the CR3BP](<./Periodic orbits in the CR3BP.ipynb>), we need to add the Poincare' phasing condition and search, instead for a solution to the system:
$$
\left\{
\begin{array}{l}
\underline{ \mathbf x}\left(T_0 + \delta T; \underline{ \mathbf x}_0 + \delta\underline{\mathbf x}\right) - \underline{ \mathbf x}_0 - \delta\underline{\mathbf x}= \mathbf 0 \\ 
\delta \underline{\mathbf x} \cdot \underline{\mathbf f} (\underline{ \mathbf x}_0) = 0
\end{array}
\right.
$$
this (both equations) is our $\underline{{\mathbf G}} = \underline{\mathbf 0}$ with $\delta T$ being our continuation parameter. We now apply the pseudo arc-length condition to avoid issues when the solution curve folds for increasing periods. We get:
$$
(2) \qquad
\left\{
\begin{array}{l}
\underline{ \mathbf x}\left(T_0 + \delta T; \underline{ \mathbf x}_0 + \delta\underline{\mathbf x}\right) - \underline{ \mathbf x}_0 - \delta\underline{\mathbf x}= \mathbf 0 \\ 
\delta \underline{\mathbf x} \cdot \underline{\mathbf f} (\underline{ \mathbf x}_0) = 0 \\
\mathbf \tau \cdot [\delta \underline{ \mathbf x}, \delta T] = \delta s
\end{array}
\right.
$$

where $\mathbf \tau$ is found differentiating  with respect to $s$ our $\underline{{\mathbf G}}$ considering both $\delta\underline{\mathbf x}(s)$ and $\delta T(s)$ as a function of $s$ and then applying a normalization.

### Predictor
If we expand in Taylor series the relation (2) we get:
$$
\left\{
\begin{array}{l}
(\underline{\underline{\mathbf \Phi}}-\underline{\underline{\mathbf I}})\delta\underline{\mathbf x} + \underline{\mathbf f} \delta T = \underline{\mathbf 0}\\ 
\delta \underline{\mathbf x} \cdot \underline{\mathbf f} (\underline{ \mathbf x}_0) = 0 \\
\mathbf \tau \cdot [\delta \underline{ \mathbf x}, \delta T] = \delta s
\end{array}
\right.
$$
which for the basis to get a first guess (predictor) on the new continued state. 

It is convenient to assemble the following matrix:
$$
\underline{\underline{\mathbf A}} = 
\left[
\begin{array}{cc}
\underline{\underline{\mathbf \Phi}}-\underline{\underline{\mathbf I}} & \underline {\mathbf f} \\
\underline {\mathbf f}^T & 0 \\
\underline {\mathbf \tau}_x^T & \tau_T
\end{array}
\right]
$$
then, the predictor scheme, becomes:

In [9]:
# Single iteration from ic, t_final, ds
def predictor(x0, T0, ds):
    # 1 - We compute the state transition matrix Phi (integrating the full EOM for t_final)
    ta.time = 0.
    ta.state[:] = x0.tolist() + ic_var
    out = ta.propagate_until(T0)
    Phi = ta.state[6:].reshape((6,6))
    # 2 - We compute the dynamics f (at zero, but for periodic orbits this is the same at T)
    state_dict = {"x":x0[0], "y":x0[1], "z":x0[2], "px":x0[3], "py":x0[4], "pz":x0[5]}
    f_dyn = np.array([hy.eval(f[i], state_dict, [mu]) for i in range(6)]).reshape((-1,1))
    # 3 - Assemble the matrix A
    A = np.concatenate((Phi-np.eye(6), f_dyn.T))
    A = np.concatenate((A,np.insert(f_dyn,-1,0).reshape((-1,1))), axis=1)
    # 4 - Compute the tangent vector tau
    tauT = 1
    taux = - np.linalg.inv(A[:,:6].T@A[:,:6])@(A[:,:6].T@A[:,-1]) * tauT
    norm = np.sqrt(taux@taux + tauT*tauT)
    tauT/=norm
    taux/=norm
    # 5 - Add to the matrix A
    tmp = np.insert(taux,6,tauT).reshape(1,-1)
    A = np.concatenate((A, tmp))
    # 6 - Compute the b vector
    b = np.zeros((8,1))
    b[7,0] = ds
    # 6 - Predict an initial guess
    y = np.linalg.inv(A.T@A)@(A.T@b)
    return y, taux, tauT

Lets instantiate our periodic solution. It is the same Lyapunov orbit we have used in the example [Continuation of Periodic Orbits in the CR3BP](<./Periodic orbits in the CR3BP.ipynb>). 

In [10]:
# These conditions correspond to a Lyapunov orbit closeto the L2 point
ic = np.array([-8.3660628427188066e-01, 6.8716725011222035e-05, 0.0000000000000000e+00, -2.3615604665605682e-05, -8.3919863043620713e-01, 0.0000000000000000e+00])
t_final=2.6915996001673941e+00
mu=0.01215057
ta.pars[0] = mu
# Reset the state
ta.time = 0
ta.state[:] = ic.tolist() + ic_var
# Time grid
t_grid = np.linspace(0, t_final, 2000)

In [11]:
# Go ...
start_time = time.time()
out = ta.propagate_grid(t_grid)
print("--- %s seconds --- to propagate" % (time.time() - start_time))

--- 0.0027968883514404297 seconds --- to propagate


In [12]:
# Pseudo arc-length
ds = 1e-3
# We call the predictor
predicted, taux, tauT = predictor(ic, t_final, ds)

# Corrector
We now want to find  $\underline{\mathbf y} = [\delta \underline{\mathbf{x}}, \delta T]$ so that the system of equations (2), here reported for cenvenience, is satisfied:
$$
\qquad
\left\{
\begin{array}{l}
\underline{ \mathbf x}\left(T_0 + \delta T; \underline{ \mathbf x}_0 + \delta\underline{\mathbf x}\right) - \underline{ \mathbf x}_0 - \delta\underline{\mathbf x}= \mathbf 0 \\ 
\delta \underline{\mathbf x} \cdot \underline{\mathbf f} (\underline{ \mathbf x}_0) = 0 \\
\mathbf \tau \cdot [\delta \underline{ \mathbf x}, \delta T] = \delta s
\end{array}
\right.
$$
and can start from the guess returned by the predictor step.

Different methods can be used to solve the overdetermined system of equations above (which admits a unique solution), we here use the least-square method implemented in scipy and also give our own implementation of a Newton based method.

First, we need to define a function that checks, for some $\underline{\mathbf y} = [\delta \underline{\mathbf{x}}, \delta T]$, whether the equations (2) are satistied.

In [13]:
def full_system(y, taux, tauT, ds):
    dx = deepcopy(y[:6])
    dT = deepcopy(y[6])
    x = ic + dx.T
    propagation_time = t_final + dT
    # 1 - We compute the state at propagation_time
    ta.time = 0.
    ta.state[:] = x.tolist() + ic_var
    out = ta.propagate_until(propagation_time)
    # 2 - We compute the dynamics f (at zero, but for periodic orbits this is the same at T)
    state_dict = {"x":ic[0], "y":ic[1], "z":ic[2], "px":ic[3], "py":ic[4], "pz":ic[5]}
    f_dyn0 = np.array([hy.eval(f[i], state_dict, [mu]) for i in range(6)]).reshape((-1,1))
    # 3 - Error on the state
    state_err = ta.state[:6] - x
    # Error on Poincare phasing
    poin_err = dx@f_dyn0
    # Error on pseudo arc-length
    pseudo_err = taux@dx+tauT*dT - ds
    # 4 - Assemble return value
    retval = np.insert(state_err,6, poin_err)
    retval = np.insert(retval,7, pseudo_err)
    return retval

... we can then call the scipy.optimze.least_squares method that will estimate gradients numerically.

In [14]:
# A least squared corrector
def corrector_ls(y, taux, tauy, ds, tol = 1e-12):
    sol = least_squares(lambda x: full_system(x, taux, tauT, ds), y, ftol = tol)
    return sol.x

In [15]:
corrected_ls = corrector_ls(predicted[:,0], taux, tauT, ds, tol=1e-14)

Here we implement our own Newton based method making use of the variational equations to compute the necessary gradient.

In [16]:
def corrector_Newton(y, taux, tauy, ds, tol=1e-12, max_iter=100, verbose = False):  
    flag_tol = False
    
    curr_dx = deepcopy(y[:6])
    curr_dT = deepcopy(y[6])
    
    # Linearization point
    curr_x = ic + curr_dx.T
    curr_T = t_final + curr_dT
    
    for i in range(max_iter):
        # 1 - We compute the state transition matrix Phi (integrating the full EOM from curr_x for t_final)
        ta.time = 0.
        ta.state[:] = curr_x.tolist() + ic_var
        out = ta.propagate_until(curr_T)
        Phi = ta.state[6:].reshape((6,6))
        # 2 - We compute the dynamics f (at the initial state)
        state_dict = {"x":ic[0], "y":ic[1], "z":ic[2], "px":ic[3], "py":ic[4], "pz":ic[5]}
        f_dyn0 = np.array([hy.eval(f[i], state_dict, [mu]) for i in range(6)]).reshape((-1,1))
        # 3 - We compute the dynamics f (at curr_T)
        state_dict = {"x":ta.state[0], "y":ta.state[1], "z":ta.state[2], "px":ta.state[3], "py":ta.state[4], "pz":ta.state[5]}
        f_dynT = np.array([hy.eval(f[i], state_dict, [mu]) for i in range(6)]).reshape((-1,1))
        # 4 - Assemble the function (the full nonlinear system) value at the current point
        state_err = ta.state[:6] - curr_x
        # Error on Poincare phasing
        poin_err = curr_dx@f_dyn0
        # Error on pseudo arc-length
        pseudo_err = taux@curr_dx+tauT*curr_dT - ds
        b = np.zeros((8,1))
        b[:6,0] = - state_err
        b[6,0] = - poin_err
        b[7,0] = - pseudo_err
        if verbose:
            print(np.linalg.norm(state_err), poin_err, pseudo_err)
        toterror = np.abs(np.linalg.norm(state_err)) + np.abs(poin_err) + np.abs(pseudo_err)
        if toterror < tol:
            flag_tol = True
            break
        # 5 - Assemble the matrix A (gradient)
        A = np.concatenate((Phi-np.eye(6), f_dynT.T))
        A = np.concatenate((A,np.insert(f_dynT,-1,0).reshape((-1,1))), axis=1)
        # add the tau row
        tmp = np.insert(taux,6,tauT).reshape(1,-1)
        A = np.concatenate((A, tmp))
        
        # Solve for new y = [dx, dT]
        y = (np.linalg.inv(A.T@A)@A.T@b).reshape(-1,)
        curr_dx += y[:6]
        curr_dT += y[6]
        curr_x = ic + curr_dx.T
        curr_T = t_final + curr_dT
    if verbose:
        if flag_tol:
            print("Exit condition - Accuracy")
        else:
            print("Exit condition - Maximum Iterations")
        print("Accuracy achieved: ", toterror)
        print("Iterations: ", i)
    return np.insert(curr_dx.T, 6, curr_dT)

In [17]:
corrected_N = corrector_Newton(predicted[:,0], taux, tauT, ds, tol=1e-14)

We now assess if the system (2) is actually solved, and to what precision, in both cases:

In [18]:
errN = full_system(corrected_N, taux, tauT,ds)
err_ls = full_system(corrected_ls, taux, tauT,ds)
print("Error for the Newton method:", np.linalg.norm(errN))
print("Error for the Least-squares method:", np.linalg.norm(err_ls))

Error for the Newton method: 9.086202142573234e-15
Error for the Least-squares method: 7.326917966640977e-13


And check the CPU time:

In [22]:
%timeit corrector_Newton(predicted[:,0], taux, tauT, ds, tol=1e-14)

22.7 ms ± 6.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit corrector_ls(predicted[:,0], taux, tauT, ds, tol=1e-14)

38 ms ± 8.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Note that this CPU timing comparison is not fair, as the least square solver is still integrating the variational equations, but not using the associated gradient information.